In [ ]:
from boto3sts import credentials as creds
import boto3
import os

In [ ]:
# oidc agent profile already configured in this server 
!oidc-gen -l

In [ ]:
# oidc-agent environment variables
!oidc-keychain

In [ ]:
os.environ['OIDC_SOCK']="/tmp/oidc-agent-service/0/oidc-agent.sock"

In [ ]:
# to access and manage object storage buckets anbnd files a token with audience "object" is required 
# AUD_TOKEN
!oidc-token --aud=object infncloud

In [ ]:
os.environ['AUD_TOKEN']="CHANGE AUD_TOKEN"

In [ ]:
!env | grep AUD_TOKEN

In [ ]:
## link where to check the token content https://jwt.io/

In [ ]:
# Get the ACCESS_KEY, SECRET_KEY and SESSION_TOKEN
sts_client = boto3.client('sts',
    endpoint_url="https://rgw.cloud.infn.it",
    region_name=''
    )

In [ ]:
response = sts_client.assume_role_with_web_identity(
    RoleArn="arn:aws:iam:::role/IAMaccess",
    RoleSessionName="CHANGE_NAME",                 # Name you prefer
    DurationSeconds=3600,
    WebIdentityToken=os.getenv('AUD_TOKEN'))

In [ ]:
# print(response)

In [ ]:
print ("ACCESS_KEY:",response['Credentials']['AccessKeyId'])
print ("SECRET_KEY:", response['Credentials']['SecretAccessKey'])
print ("SESSION_TOKEN:", response['Credentials']['SessionToken'])

In [ ]:
s3client = boto3.client('s3',
    aws_access_key_id = response['Credentials']['AccessKeyId'],
    aws_secret_access_key = response['Credentials']['SecretAccessKey'],
    aws_session_token = response['Credentials']['SessionToken'],
    endpoint_url="https://rgw.cloud.infn.it",
    region_name='default')

In [ ]:
# Show files in a bucket

for key in s3client.list_objects(Bucket='scratch')['Contents']:
        print(key['Key'])

In [ ]:
# Attributes of s3 object
print(dir(s3client))

In [ ]:
# The current directory
os.getcwd()

In [ ]:
# To upload a file to RGW bucket
s3client.upload_file('LOCAL_PATH_OF_FILE', 'BUCKET', 'PATH_WHERE_TO_UPLOAD')

In [ ]:
# Download a file from RGW to the current workarea specifying your bucket name
s3client.download_file('BUCKET','PATH_FILE_TO_DOWNLOAD', 'PATH_WHERE_TO_DOWNLOAD')

In [ ]:
# Content of the workarea
for entry in os.scandir():
    print(entry)

In [ ]:
for entry in os.scandir('/workarea'):
    print(entry)